In [1]:
import spacy
from pptx import Presentation
import nltk, collections
import re

In [2]:
from collections import defaultdict

master_index = defaultdict(list)

In [3]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

Extract all the words in a ppt while retaining slide numbers

In [4]:
filepath = str(input('Enter your filepath (or filename if in the same folder): ')) #filepath
pres = Presentation(filepath)
slideCount = 0
indexes = {}

In [5]:
for slide in pres.slides:
    slideCount += 1
    words_in_slide = [slideCount]
#     print('slide number', slideCount)
    for shape in slide.shapes:
        if (shape.has_text_frame):
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    # print(run.text)
                    # print("    ---- New Line ----    ")
                    words_in_slide.append(run.text)
    indexes[slideCount] = words_in_slide

In [ ]:
for i in range(len(indexes)):
    print(i+1, " - - - ", indexes[i+1])

For each slide's words, get the most important words

In [17]:
unwanted_terms = {'\t','(',')','=','.','-',':'}
numbers = set([i for i in '0123456789'])
# alphabets = set([i for i in 'abcdefghijklmnopqrstuvwx'])

def remove_all_instances(main_text, old_term, replacement=""):
    s = main_text.replace(old_term, replacement)
    return s

For now, we're only using 1 slide but once we have it down, we can put it into a loop

In [ ]:
#DONT RUN

slideNumber = 5
slide = indexes[slideNumber]
slideText = " ".join(slide[1:]).strip()

for term in unwanted_terms:
    slideText = remove_all_instances(slideText,term)

slideText
# slideText_without_unwanted = slideText.replace("\t","")
# slideText_without_unwanted

In [ ]:
#DONT RUN

word_tokens = nltk.word_tokenize(slideText)
print(word_tokens)

In [ ]:
#DONT RUN

words_without_stops = [w for w in word_tokens if w not in stop_words]
# words_without_stops

In [8]:
def remove_words_with_n_characters(tokenised_list, n_characters = 1):
    new_list = tokenised_list.copy()
    for word in tokenised_list:
        if (len(word) == n_characters):
#             print(word, " --- ", len(word))
#             print(len(word))
#             print(word)
            new_list.remove(word)
#             print(words_without_stops)
#         else:
#             print("NOT REJECTED: ",word, " --- ", len(word))
    return new_list


In [ ]:
#DONT RUN

words_without_singularsStops = remove_words_with_n_characters()

words_without_singularsStops = [x.lower() for x in words_without_singularsStops]

print(words_without_singularsStops)

In [13]:
#DONT NEED TO RUN until end maybe

frequencies = collections.Counter() 
for w in words_without_singularsStops:
    frequencies[w] += 1
frequencies

NameError: name 'words_without_singularsStops' is not defined

In [ ]:
#DONT NEED TO RUN until end maybe

bigram = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(words_without_singularsStops)
finder.apply_freq_filter(2)
print (finder.nbest(bigram.pmi, 30))

Put into Index by page

DefaultDict is useful in this case cause it creates a new list when the key is missing. It can be an issue if you're intending to use this as read_only, cause it will return an empty list instead of None. But since, we're not calling any values, it's fine.

In [ ]:
#DONT RUN

for word in set(words_without_singularsStops):
    master_index[word].append(slideNumber)
#     print(word, type(word))
    
print(master_index)

## Put all of that into a function:

In [18]:
def index_slide(slideNumber):
    slide = indexes[slideNumber]
    slideText = " ".join(slide[1:]).strip()

    for term in unwanted_terms:
        slideText = remove_all_instances(slideText,term)
    
    for number in numbers:
        slideText = remove_all_instances(slideText,number)
    
    word_tokens = nltk.word_tokenize(slideText)
    
    words_without_stops = [w for w in word_tokens if w not in stop_words]
    
    words_without_singularsStops = remove_words_with_n_characters(words_without_stops)

    words_without_singularsStops = [x.lower() for x in words_without_singularsStops]
    
    for word in set(words_without_singularsStops):
        master_index[word].append(slideNumber)

In [19]:
for i in range(len(indexes)):
#     print(i+1, " - - - ", indexes[i+1])
    index_slide(i+1)

In [ ]:
master_index

In [20]:
output_str=""
with open('index.txt','w') as f:    
        for elem in sorted(master_index):
            try:
                f.write(str(elem) + " :: " + str(master_index[elem]) + "\n")
            except UnicodeEncodeError:
                print(f'{elem}, found on pages {master_index[elem]} was not encoded')

2ℓ, found on pages [69] was not encoded
2𝐴, found on pages [69] was not encoded
2𝑘𝑇, found on pages [69] was not encoded
2𝜋𝑘𝑇, found on pages [69] was not encoded
3𝜋, found on pages [69] was not encoded
3𝜋𝜇𝐷, found on pages [69] was not encoded
4αη, found on pages [183] was not encoded
e𝑘, found on pages [162, 162] was not encoded
αη, found on pages [183] was not encoded
μm, found on pages [140, 140] was not encoded
πd, found on pages [183, 183] was not encoded
π𝐷, found on pages [91, 91] was not encoded
ρg, found on pages [47, 49, 50, 47, 49, 50] was not encoded
ρu, found on pages [47, 49, 50, 47, 49, 50] was not encoded
∇𝐷∇𝑛, found on pages [104, 104] was not encoded
∇𝑛, found on pages [147, 147] was not encoded
−4𝛼𝐿𝜂, found on pages [184] was not encoded
−∇, found on pages [104, 104] was not encoded
−𝛼𝐿𝜂, found on pages [184] was not encoded
−𝜇𝑚, found on pages [17, 19, 29, 32, 33, 17, 19, 29, 32, 33] was not encoded
, found on pages [132, 132] was not encoded
, found on pages 